In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
#import_original_function
import sys
sys.path.append("../")
from function.data_processing import addtional_process


In [20]:
#configファイルの読み込み
import configparser
config_ini = configparser.ConfigParser()
config_ini.read('../config/config.ini', encoding='utf-8')

['../config/config.ini']

In [21]:
#read_file_congfig
test_file       = "." + config_ini["INPUT_FILE"]["test"]
train_file      = "." + config_ini["INPUT_FILE"]["train"]
deginition_file = "." + config_ini["CONFIG_FILE"]["deginition"]

#### 1.import data

In [23]:
#データの成型方法を読込
df_deginition  = pd.read_excel(deginition_file,index_col=0, dtype = 'object')
df_deginition  = df_deginition.query('採否 == "◎"|採否 == "〇"| 採否 == "△"')

#学習に使うカラムを選択
target_columns = df_deginition["本番データ特徴量名"].values.tolist()

#ダミー変数化するカラムを選択
# dummy_columns = df_deginition.query("ダミー変数化 == '〇'")["本番データ特徴量名"].values.tolist()


In [24]:
# 対象のCSVデータの読込
df_train_raw = pd.read_csv(train_file , header=0,sep=',' , dtype = 'object')

# 対象カラムを絞り込み
df_train = df_train_raw[target_columns]

# 抽出結果を表示
display(df_train.head(2))

,money_room,building_status,building_type,unit_count,building_structure,floor_count,year_built,room_floor,dwelling_unit_window_angle,room_count,...,walk_distance2,flg_new,room_kaisuu,snapshot_window_angle,madori_number_all,madori_kind_all,parking_kubun,genkyo_code,convenience_distance,super_distance
0,38000,1,3,22,1,2,200210,2,5,1,...,NaN,0,2,5,1,20,1,3,239,1280
1,103000,1,1,NaN,4,3,197612,1,5,2,...,400,0,1,5,2,30,3,2,399,434


#### 2.replace data_type and NA

In [25]:
#前処理（データ型変換、補完）

# 欠損個所をNAという文字列で置換
df_train = df_train.fillna("NA")
# df_test = df_test.fillna("NA")

for idx ,raw  in df_deginition.iterrows():
    #データの変換対象、変換方法を確認
    col = raw["本番データ特徴量名"]
    complement_value = raw["欠損補完方法"]
    type = raw["データ型"]

    #補完値が空でなければ補完
    if complement_value != np.nan : 
        try:
            df_train[col] = df_train[col].replace("NA",complement_value)
        except Exception as e:
            print("補完",col,complement_value,type)
            print(f"エラーが発生しました: {e}")
        # try:
        #     df_test[col] = df_test[col].replace("NA",complement_value)
        # except Exception as e:
        #     print("補完",col,complement_value,type)
        #     print(f"エラーが発生しました: {e}")

    #データ型変換
    try:
        df_train[col] = df_train[col].astype(type)
    except Exception as e:
        print("置換",col,complement_value,type)
        print(f"エラーが発生しました: {e}")
    # try:
    #     df_test[col]  = df_test[col].astype(type)
    # except Exception as e:
    #     print("置換",col,complement_value,type)
    #     print(f"エラーが発生しました: {e}")



#### 2-3.additional_processing

In [26]:
#特殊前処理
df_train = addtional_process(df_train)
# df_test  = addtional_process(df_test)

In [27]:
#train_dataのみ追加処理

# 東京都（addr1_1=13）以外の100万円以上のデータを中央値の52000円で置換
df_train.loc[(df_train['addr1_1'] != '13') & (df_train['money_room'] >= 1000000), 'money_room'] = df_train['money_room'].median()

df_train["unit_area"] = df_train["unit_area"].fillna(0)
df_train["unit_area"] = df_train["unit_area"].astype(type)

# # df_train['unit_area']が300以上、df_train['money_room']が50万円以下の物件を中央値で置換
df_train.loc[(df_train['unit_area'] >= 300) & (df_train['money_room'] < 500000), 'unit_area'] = df_train['unit_area'].median()

# year_builtが70以上の物件を、year_builtの中央値で置換
df_train.loc[df_train['year_built'] > 70, 'year_built'] = df_train['year_built'].median()

# floor_countが70以上の物件を、floor_countの中央値で置換
df_train.loc[df_train['floor_count'] > 64, 'floor_count'] = df_train['floor_count'].median()


In [29]:
len(df_train.query("money_room < 500000"))

584272

In [31]:
len(df_train.query("money_room > 500000"))

222

In [30]:
len(df_train)

584507

In [33]:
# コサイン類似度を計算
similarity_matrix = cosine_similarity(df_train)
# 類似度が0.9以上のペアを検出
similarity_th = 0.9
money_room_th = 500000
similar_indices = np.argwhere((similarity_matrix >= similarity_th) & (similarity_matrix < 1.0))


MemoryError: Unable to allocate 2.49 TiB for an array with shape (584507, 584507) and data type float64

In [ ]:
# 重複削除のために、各ペアのうち一方のみを保持
to_remove = set()
for i, j in similar_indices:
    if i < j and df_train.loc[j,"moneyroom"] < money_room_th:
        to_remove.add(j)

ValueError: could not convert string to float: '※専有面積にトランクルーム面積0.25平米を含む ポーチ面積：3.53平米（約3.96坪） 大規模修繕の予定：検討中'

In [ ]:
# 類似データを削除 ※money_room <50万に限る
print("重複削除後のデータ:",len(selected_df))
df_deduped = selected_df.drop(list(to_remove)).reset_index(drop=True)
print("重複削除後のデータ:",len(df_deduped))